In [162]:
from numpy import linalg as LA
import numpy as np
from clarifai.rest import ClarifaiApp
from clarifai.rest import Image as ClImage
app = ClarifaiApp(api_key='df477c72fbb246b99921528fa25c54bd')
#Face embedding model
model = app.models.get("d02b4508df58432fbb84e800597b8959")
image1 = ClImage(file_obj=open('HKR.jpg', 'rb'))
image2 = ClImage(file_obj=open('SRK.jpg', 'rb'))
result1=model.predict([image1])
result2=model.predict([image2])

outputs1=result1['outputs'][0]
outputs2=result2['outputs'][0]
#print(list(outputs.keys()))

data1=np.array(outputs1['data']['regions'][0]['data']['embeddings'][0]['vector'])
data2=np.array(outputs2['data']['regions'][0]['data']['embeddings'][0]['vector'])
#print(list(data.keys()))
print(LA.norm(data1-data2))
image
print(data1)
print(data2)






ConnectionError: HTTPSConnectionPool(host='api.clarifai.com', port=443): Max retries exceeded with url: /v2/models?page=1&per_page=20 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002218FB0C4E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [ ]:
import cv2
img=cv2.imread('HKR.jpg',0)
cv2.imshow('image',img)

In [ ]:
file_obj=open('HKR.jpg', 'rb')
for i in file_obj:
    print(i)

In [ ]:
import os
import cv2
from numpy import linalg as LA
import numpy as np
from clarifai.rest import ClarifaiApp
from clarifai.rest import Image as ClImage

def grab():
    app = ClarifaiApp(api_key='df477c72fbb246b99921528fa25c54bd')
    model = app.models.get("d02b4508df58432fbb84e800597b8959")
    cap=cv2.VideoCapture(0)
    while(True):
        ret, frame = cap.read()
        cv2.imshow('camera',frame)
        key=cv2.waitKey(1) & 0xFF
        if key==13:
            image_name="face.jpg"
            cv2.imwrite(image_name,frame)
            break

    cap.release()
    cv2.destroyAllWindows()
    image = ClImage(file_obj=open(image_name, 'rb'))

    result=model.predict([image])
    outputs=result['outputs'][0]
    data=np.array(outputs['data']['regions'][0]['data']['embeddings'][0]['vector'])
    image=None
    os.remove(image_name)
    return data


In [ ]:
import json
import numpy as np

with open('jsonResp/Jacques_Chirac_0005.json') as f:
    face_feature_json=json.load(f)
    
data=np.array(face_feature_json['outputs'][0]['data']['regions'][0]['data']['embeddings'][0]['vector'])
    
    

In [ ]:
max(data)

In [ ]:
('f296eb46ef684787b147988e1796b355'=='ec1740642c83478392e7b8735c43c630')

In [11]:
import os
import json
import random
import numpy as np
input_size=8000
file_list = os.listdir('jsonResp')
input_vecs=np.zeros((input_size*2,1024))
labels=np.zeros((input_size*2,1))
cntr=0
i=0
while i <=input_size:
    
    with open('jsonResp/'+file_list[i]) as f:
        face1_json=json.load(f)
    face1_vec=np.array(face1_json['outputs'][0]['data']['regions'][0]['data']['embeddings'][0]['vector'])
    for j in range(i+1,i+9):
        try:
            with open('jsonResp/'+file_list[j]) as f:
                face2_json=json.load(f)
            face2_vec=np.array(face2_json['outputs'][0]['data']['regions'][0]['data']['embeddings'][0]['vector'])
            input_vecs[cntr,:]=abs(face1_vec-face2_vec)
            if file_list[i][:-10]==file_list[j][:-10]:
                labels[cntr]=1
            #print([i, j, labels[cntr]])
            cntr+=1
        except:
            continue
        
    while file_list[i][:-10]==file_list[j][:-10]:
        j+=1
    for k in random.sample(list(range(i))+list(range(j,input_size+1000)),8):
        try:
            with open('jsonResp/'+file_list[k]) as f:
                face2_json=json.load(f)
            face2_vec=np.array(face2_json['outputs'][0]['data']['regions'][0]['data']['embeddings'][0]['vector'])
            input_vecs[cntr,:]=abs(face1_vec-face2_vec)
            if file_list[i][:-10]==file_list[k][:-10]:
                labels[cntr]=1
            #print([i, k, labels[cntr]])
            cntr+=1
        except:
            continue
        
    i=j

input_vecs=input_vecs[:cntr+1]
labels=labels[:cntr+1]
shuff_ind=np.arange(len(labels))
np.random.shuffle(shuff_ind)
labels=labels[shuff_ind]
input_vecs=input_vecs[shuff_ind]

labels_train=labels[:int(0.85*len(labels))]
labels_test=labels[int(0.85*len(labels))+1:]
input_vecs_train=input_vecs[:int(0.85*len(labels))]
input_vecs_test=input_vecs[int(0.85*len(labels))+1:]



    

In [164]:
"""import keras
labels_train_onehot=keras.utils.to_categorical(labels_train,num_classes=2)
labels_test_onehot=keras.utils.to_categorical(labels_test,num_classes=2)
"""

In [12]:
labels_train.shape

(9763, 1)

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model=Sequential()
model.add(Dense(800,input_shape=(1024,),activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10,activation='relu'))
model.add(Dense(3,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [30]:
from keras.optimizers import SGD
sgd=SGD(lr=0.03,decay=0.0001,momentum=0.5,nesterov=False)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [31]:
model.fit(input_vecs_train,labels_train,batch_size=1,epochs=15)

Epoch 1/15
9763/9763 [==============================] - 69s 7ms/step - loss: 0.1276 - acc: 0.9514
Epoch 2/15
9763/9763 [==============================] - 61s 6ms/step - loss: 0.0370 - acc: 0.9870
Epoch 3/15
9763/9763 [==============================] - 61s 6ms/step - loss: 0.0259 - acc: 0.9905
Epoch 4/15
9763/9763 [==============================] - 61s 6ms/step - loss: 0.0193 - acc: 0.9934
Epoch 5/15
9763/9763 [==============================] - 64s 7ms/step - loss: 0.0165 - acc: 0.9948
Epoch 6/15
9763/9763 [==============================] - 63s 6ms/step - loss: 0.0132 - acc: 0.9957
Epoch 7/15
9763/9763 [==============================] - 61s 6ms/step - loss: 0.0095 - acc: 0.9972
Epoch 8/15
9763/9763 [==============================] - 61s 6ms/step - loss: 0.0071 - acc: 0.9977
Epoch 9/15
9763/9763 [==============================] - 67s 7ms/step - loss: 0.0082 - acc: 0.9974
Epoch 10/15
9763/9763 [==============================] - 66s 7ms/step - loss: 0.0070 - acc: 0.9977
Epoch 11/15
9763/97

In [32]:
model.evaluate(input_vecs_test,labels_test)

1723/1723 [==============================] - 0s 244us/step


[0.06893232927485507, 0.9901334881021474]

In [33]:
import h5py
model.save('verify_face.h5')

In [35]:
import os
from numpy import linalg as LA
import numpy as np
import cv2
from numpy import linalg as LA
import numpy as np
from clarifai.rest import ClarifaiApp
from clarifai.rest import Image as ClImage
from keras.models import load_model

def grab_face_vector():
    app = ClarifaiApp(api_key='df477c72fbb246b99921528fa25c54bd')
    model = app.models.get("d02b4508df58432fbb84e800597b8959")
    cap=cv2.VideoCapture(0)
    while(True):
        ret, frame = cap.read()
        cv2.imshow('camera',frame)
        key=cv2.waitKey(1) & 0xFF
        if key==13:
            image_name="face.jpg"
            cv2.imwrite(image_name,frame)
            break

    cap.release()
    cv2.destroyAllWindows()
    image = ClImage(file_obj=open(image_name, 'rb'))
    result=model.predict([image])
    outputs=result['outputs'][0]
    data=np.array(outputs['data']['regions'][0]['data']['embeddings'][0]['vector'])
    image=None
    os.remove(image_name)
    return data

inp1=input("Please enter '1' to take the first picture")

if inp1=='1':
    data1=grab_face_vector()
else:
    print("Wrong input...Try again")

if inp1=='1':
    inp2=input("Please enter '2' to compare new pictures to the first picture")
    if inp2=='2':
        data2=grab_face_vector()
        input_vec=abs(data1-data2)
        input_vec=input_vec.reshape(1,1024)
        model=load_model('verify_face.h5')
        result=model.predict(input_vec)
        print(result)
    else:
         print("Wrong input...Try again")   
        


Please enter '1' to take the first picture1
Please enter '2' to compare new pictures to the first picture2
[[0.99933213]]


In [5]:
input_vec=input_vec.reshape(1,1024)
input_vec.shape

(1, 1024)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'face.jpg'